In [ ]:
# 기준 4가지 별 이미지 유사도 비교 - 최사위 3개 이미지 출력

import cv2
import numpy as np
import os

def calculate_histogram_similarity(hist1, hist2, method):
    # 유사도 계산
    similarity = cv2.compareHist(hist1, hist2, method)
    return similarity

def get_histogram(image_path):
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv_image], [0, 1], None, [50, 60], [0, 180, 0, 256])
    hist = cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
    return hist

def find_most_similar_images(target_image_path, folder_path, method, top_n=3):
    target_hist = get_histogram(target_image_path)
    distances = []

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and file_path != target_image_path:
            hist = get_histogram(file_path)
            distance = calculate_histogram_similarity(target_hist, hist, method)
            distances.append((distance, filename))

    if method in [cv2.HISTCMP_CORREL, cv2.HISTCMP_INTERSECT]:
        distances.sort(key=lambda x: x[0], reverse=True)
    else:
        distances.sort(key=lambda x: x[0])
    
    most_similar_images = [filename for _, filename in distances[:top_n]]

    return most_similar_images

target_image_path = '/Users/suntaeyeol/Desktop/coding/deep/team/K-Fashion/hist/1.jpg'
folder_path = '/Users/suntaeyeol/Desktop/coding/deep/team/K-Fashion/crop_data_origin/top_crop_image_origin/shirts'

methods = {
    'Correlation': cv2.HISTCMP_CORREL,
    'Chi-Square': cv2.HISTCMP_CHISQR,
    'Intersection': cv2.HISTCMP_INTERSECT,
    'Bhattacharyya': cv2.HISTCMP_BHATTACHARYYA
}

for method_name, method in methods.items():
    most_similar_images = find_most_similar_images(target_image_path, folder_path, method, top_n=3)
    print(f"The most similar images using {method_name} are:")
    for image in most_similar_images:
        print(image)
    print("\n")

In [ ]:
import cv2
import numpy as np
import os

def calculate_histogram_similarity(hist1, hist2, method):
    similarity = cv2.compareHist(hist1, hist2, method)
    return similarity

def get_histogram(image_path):
    image = cv2.imread(image_path)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv_image], [0, 1], None, [50, 60], [0, 180, 0, 256])
    hist = cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
    return hist

def find_most_similar_images(target_image_path, folder_path, top_n=3):
    methods = {
        'Correlation': (cv2.HISTCMP_CORREL, 1),     
        'Chi-Square': (cv2.HISTCMP_CHISQR, -1),     
        'Intersection': (cv2.HISTCMP_INTERSECT, 1), 
        'Bhattacharyya': (cv2.HISTCMP_BHATTACHARYYA, -1) 
    }

    target_hist = get_histogram(target_image_path)
    similarity_scores = []

    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and file_path != target_image_path:
            hist = get_histogram(file_path)
            score = 0
            for method_name, (method, direction) in methods.items():
                similarity = calculate_histogram_similarity(target_hist, hist, method)
                score += direction * similarity
            similarity_scores.append((score, filename))

    similarity_scores.sort(key=lambda x: x[0], reverse=True)
    most_similar_images = [filename for _, filename in similarity_scores[:top_n]]

    return most_similar_images

target_image_path = 'jpg'
folder_path = 'path'

most_similar_images = find_most_similar_images(target_image_path, folder_path, top_n=3)
print("The most similar images are:")
for image in most_similar_images:
    print(image)